In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName('branches') \
    .config("spark.jars", "/opt/spark/jars/iceberg-spark-runtime-3.5_2.12-1.6.0.jar") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hive") \
    .config("spark.sql.catalog.local.warehouse", "s3a://datalake/iceberg") \
    .getOrCreate()

#Ajuste de log WARN log para ERROR
spark.sparkContext.setLogLevel("ERROR")

### Branching and Tagging - Visão Geral

As tabelas do Iceberg matem um log de snapshots, que representão as mudanças aplicadas na tabela.
Os snapshots são fundamentais no formato de tabela Iceberg, pois eles garantem o isolamento das onsultas/queries e permitem o "time travel" entre as diferentes "versões de dados" da tabela.

Para carontrolar o tamanho dos metadados e custos de armazenamento, Iceberg fornece um gerenciamento de ciclo de vida de snapshots. Que pode ser executado via procedures como **_expire_snapshots_** para remoção de de arquivos de dados não utilizados com base nas propriedades dos snapshots.

Para possibilidar o gerenciamento de ciclo de vidade de snapshots. O Iceberg suporta Branches e Tags, Esses recursos garantem que os snapshots tenham seus proprios ciclos de vida que por sua vez podem ser controlados por politicas de retenção em nivel de Branch e tag.

fonte: https://iceberg.apache.org/docs/1.6.1/branching/

### Criar Branche e Exibir seus dados

In [ ]:
# Criar uma branch chamada dev_branch
spark.sql("ALTER TABLE iceberg.bronze.vendas CREATE BRANCH dev_branch").show()

In [ ]:
## Exibir os dados da brach dev_branch
spark.sql("SELECT * FROM iceberg.bronze.vendas.branch_dev_branch").toPandas()

### Inspecionar tabela

In [ ]:
# Listar branches
spark.sql("SELECT * FROM iceberg.bronze.vendas.refs;").toPandas()

In [ ]:
# Listar snapshots
spark.sql("SELECT * FROM iceberg.bronze.vendas.snapshots;").toPandas()

### Excluir dados da brach dev_branch

In [ ]:
## Excluir algumas linhas da brach dev_branch

spark.sql("""
    DELETE FROM iceberg.bronze.vendas.branch_init_branch
    WHERE product_line_id = '1002';
""").show()

In [ ]:
## Exibir novamente os dados da brach dev_branch

spark.sql("SELECT * FROM iceberg.bronze.sales_order.branch_dev_branch").toPandas()

In [ ]:
## Exibir os dados da brach main
spark.sql("SELECT * FROM iceberg.bronze.sales_order").toPandas()

### Criar Tags